<a href="https://colab.research.google.com/github/jonathanahc/PLN_Practicas/blob/main/Tarea_An%C3%A1lisis_de_temas_clustering_Jonathan_Herrera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gensim
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel
from itertools import product
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
documents = [
    # Política
    "El presidente anunció nuevas reformas durante la conferencia en el Congreso.",
    "Los partidos políticos debatieron sobre el futuro del sistema electoral.",
    "La oposición critica la gestión gubernamental en materia de seguridad.",
    "Miles de personas protestaron en la capital contra la nueva ley propuesta.",
    "Se firmó un acuerdo internacional para fomentar la cooperación política.",

    # Economía
    "El banco central decidió aumentar las tasas de interés para controlar la inflación.",
    "La economía del país creció un 3% durante el segundo trimestre del año.",
    "Las exportaciones de productos agrícolas aumentaron notablemente.",
    "El desempleo bajó por tercer mes consecutivo, según datos oficiales.",
    "Los inversores mostraron confianza ante las nuevas políticas económicas.",

    # Deportes
    "La selección nacional clasificó a la final del campeonato continental.",
    "El delantero estrella fue transferido al club europeo por una suma millonaria.",
    "Los fanáticos celebraron la victoria con una caravana en las calles.",
    "El equipo local perdió el partido decisivo por un gol en el último minuto.",
    "La liga anunció cambios en el reglamento para la próxima temporada.",

    # Tecnología
    "La empresa lanzó un nuevo smartphone con inteligencia artificial integrada.",
    "Se descubrió una vulnerabilidad crítica en el sistema operativo.",
    "La inversión en energías renovables incluye avances en baterías inteligentes.",
    "El uso de la automatización ha transformado la industria manufacturera.",
    "Expertos debaten sobre los riesgos éticos del desarrollo de la IA.",

    # Medio ambiente
    "El cambio climático está afectando los patrones de lluvia en la región.",
    "Un nuevo informe advierte sobre la pérdida acelerada de biodiversidad.",
    "Se implementaron miles de políticas públicas para reducir las emisiones de carbono.",
    "Miles de voluntarios participaron en una jornada de reforestación.",
    "Organizaciones internacionales exigen medidas urgentes contra la contaminación."
]


In [4]:
stop_words = set(stopwords.words('spanish'))
texts = [
    [word for word in word_tokenize(doc.lower()) if word.isalpha() and word not in stop_words]
    for doc in documents
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [8]:
def train_lda(dictionary, corpus, texts, num_topics, alpha, eta, random_state=42):

  model = LdaModel(
      corpus=corpus,
      id2word=dictionary,
      num_topics=num_topics,
      alpha=alpha,
      eta=eta,
      random_state=random_state,
      chunksize=200,
      passes=10,
      iterations=200,
      eval_every=None
  )

  log_perp = model.log_perplexity(corpus)

  cm = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
  coh = cm.get_coherence()

  return model, log_perp, coh

topic_grid = [3, 5, 7, 9]
alpha_grid = ['symmetric', 'asymmetric', 0.1, 0.3, 0.7]
eta_grid   = ['symmetric', 0.01, 0.1]

results = []
best = {"model": None, "coh": -1e9}

for k, a, b in product(topic_grid, alpha_grid, eta_grid):
  model, logp, coh = train_lda(dictionary, corpus, texts, num_topics=k, alpha=a, eta=b)
  results.append({
      "num_topics": k,
      "alpha": a,
      "beta(eta)": b,
      "log_perplexity": logp,
      "coherence_c_v": coh
    })
  if coh > best["coh"]:
    best["coh"] = coh
    best["model"] = model


df_results = pd.DataFrame(results).sort_values("coherence_c_v", ascending=False).reset_index(drop=True)
print(df_results.head(10))

   num_topics       alpha  beta(eta)  log_perplexity  coherence_c_v
0           7  asymmetric       0.01       -7.781102       0.566554
1           7  asymmetric        0.1       -6.253693       0.552234
2           7  asymmetric  symmetric       -6.096206       0.528721
3           3         0.3       0.01       -7.935901       0.520242
4           3         0.3        0.1       -6.233678       0.520242
5           3   symmetric       0.01       -7.949537       0.520242
6           3   symmetric        0.1       -6.247217       0.520242
7           3         0.1       0.01       -7.836755       0.507639
8           9  asymmetric       0.01       -7.707841       0.507566
9           3  asymmetric        0.1       -6.378564       0.500568


In [9]:
for idx, topic in best["model"].print_topics():
    print(f"Tema {idx}: {topic}")

Tema 0: 0.026*"miles" + 0.026*"anunció" + 0.026*"nuevo" + 0.013*"políticas" + 0.013*"nuevas" + 0.013*"acelerada" + 0.013*"carbono" + 0.013*"suma" + 0.013*"club" + 0.013*"públicas"
Tema 1: 0.026*"sistema" + 0.026*"partido" + 0.026*"equipo" + 0.026*"inflación" + 0.026*"interés" + 0.026*"gol" + 0.026*"último" + 0.026*"banco" + 0.026*"celebraron" + 0.026*"tasas"
Tema 2: 0.070*"nueva" + 0.070*"personas" + 0.070*"capital" + 0.070*"exigen" + 0.070*"ley" + 0.070*"organizaciones" + 0.070*"protestaron" + 0.070*"urgentes" + 0.070*"propuesta" + 0.070*"contaminación"
Tema 3: 0.135*"final" + 0.135*"campeonato" + 0.135*"selección" + 0.135*"continental" + 0.135*"nacional" + 0.135*"clasificó" + 0.001*"miles" + 0.001*"políticas" + 0.001*"nuevo" + 0.001*"voluntarios"
Tema 4: 0.065*"según" + 0.065*"bajó" + 0.065*"datos" + 0.065*"partidos" + 0.065*"mes" + 0.065*"consecutivo" + 0.065*"oficiales" + 0.065*"políticos" + 0.065*"desempleo" + 0.065*"electoral"
Tema 5: 0.088*"automatización" + 0.088*"manufacturera

In [11]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

vis = gensimvis.prepare(best["model"], corpus, dictionary)

pyLDAvis.save_html(vis, "lda_best_vis.html")

print("Visualización guardada en lda_best_vis.html")

Visualización guardada en lda_best_vis.html


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
